In [ ]:
#| default_exp inference.core
#| default_cls_lvl 3

In [ ]:
#| export
from tsfast.datasets.core import extract_mean_std_from_dls
from tsfast.data.loader import reset_model_state
from tsfast.models.layers import NormalizedModel
import warnings
from tsfast.prediction.core import PredictionCallback
import numpy as np
import torch

In [ ]:
#| export
class InferenceWrapper:
    """
    (Docstring remains the same)
    """
    def __init__(self, learner, device='cpu'):
        # (Initialization remains the same: detect callback, store its state, get main norm)
        if not hasattr(learner, 'model') or not hasattr(learner, 'dls'):
            raise TypeError("Learner object seems invalid.")
        self.device = device
        self.core_model = learner.model.to(self.device)
        self._uses_prediction_callback = False
        self._y_init_norm = None
        self._pred_cb_t_offset = 0
        for cb in learner.cbs:
            if isinstance(cb, PredictionCallback):
                if cb.norm is not None:
                    self._uses_prediction_callback = True
                    self._y_init_norm = cb.norm.to(device)
                    self._pred_cb_t_offset = cb.t_offset
                    print(f"InferenceWrapper: Detected PredictionCallback(t_offset={self._pred_cb_t_offset}).")
                    break
                else: warnings.warn("Found PredictionCallback, but norm uninitialized.")
        mean, std = extract_mean_std_from_dls(learner.dls)
        if mean is None or std is None: raise ValueError("Could not extract main mean/std.")
        self.norm_model = NormalizedModel(self.core_model, mean, std).to(self.device)
        self.norm_model.eval()
        self.expected_feature_dim = self.norm_model.mean.shape[-1]

    # --- Helper Functions (Keep these separate for clarity) ---
    def _normalize_y_init(self, np_output_init: np.ndarray) -> torch.Tensor:
        # ... (same as before)
        if self._y_init_norm is None: raise RuntimeError("y_init normalizer not found.")
        y_init_tensor = torch.from_numpy(np_output_init).float().to(self.device)
        return self._y_init_norm.normalize(y_init_tensor)

    def _prepare_input_3d(self, np_ar: np.ndarray, name: str) -> np.ndarray:
        # ... (same as before)
        if np_ar.ndim == 1: np_ar = np.expand_dims(np_ar, axis=(0, -1))
        elif np_ar.ndim == 2: np_ar = np.expand_dims(np_ar, axis=0)
        elif np_ar.ndim != 3 or np_ar.shape[0] != 1:
             raise ValueError(f"{name} must be 1D/2D/3D(batch=1). Got {np_ar.shape}")
        return np_ar

    def _adjust_seq_len(self, np_arr: np.ndarray, target_len: int, name: str) -> np.ndarray:
        # ... (same as before)
        current_len = np_arr.shape[1]
        if current_len == target_len: return np_arr
        if current_len < target_len:
            pad_width = target_len - current_len
            return np.pad(np_arr, ((0, 0), (0, pad_width), (0, 0)), mode='constant', constant_values=0)
        # current_len > target_len
        warnings.warn(f"Truncating {name} len from {current_len} to {target_len}.", UserWarning)
        return np_arr[:, :target_len, :]
    # -----------------------------------------------------------

    def inference(self, np_input: np.ndarray, np_output_init: np.ndarray = None) -> np.ndarray:
        # 1. Prepare u (np_input)
        np_input = self._prepare_input_3d(np_input, "np_input")
        u_features = np_input.shape[-1]
        input_seq_len = np_input.shape[1]

        # 2. Prepare final numpy input based on detected mode
        final_np_input = None
        if self._uses_prediction_callback:
            if np_output_init is None: raise ValueError("PredictionCallback model requires np_output_init.")
            np_output_init = self._prepare_input_3d(np_output_init, "np_output_init")

            effective_len = input_seq_len - self._pred_cb_t_offset
            if effective_len <= 0: raise ValueError(f"Input len too short for offset {self._pred_cb_t_offset}.")

            np_input_adj = np_input[:, self._pred_cb_t_offset:, :]
            np_y_init_adj = self._adjust_seq_len(np_output_init, effective_len, "y_init (for offset)")
            norm_y_init_np = self._normalize_y_init(np_y_init_adj).cpu().numpy()
            final_np_input = np.concatenate((np_input_adj, norm_y_init_np), axis=-1)

        elif np_output_init is None: # No callback, no y_init -> Check simulation
            if u_features != self.expected_feature_dim:
                 raise ValueError(f"No y_init. Expected {self.expected_feature_dim} features, got {u_features}.")
            final_np_input = np_input # Simulation case

        else: # No callback, but y_init provided -> Check dimensions
             np_output_init = self._prepare_input_3d(np_output_init, "np_output_init")
             y_init_features = np_output_init.shape[-1]

             if u_features == self.expected_feature_dim: # Matches u alone
                 warnings.warn("Input `u` matches expected features. Ignoring provided y_init.", UserWarning)
                 final_np_input = np_input # Simulation case
             elif u_features + y_init_features == self.expected_feature_dim: # Matches u + y_init
                 np_y_init_adj = self._adjust_seq_len(np_output_init, input_seq_len, "y_init") # Adjust raw y_init length
                 final_np_input = np.concatenate((np_input, np_y_init_adj), axis=-1) # Dataloader Prediction case
             else: # Dimension mismatch
                 raise ValueError(f"Dim mismatch. Expected {self.expected_feature_dim}. "
                                  f"u:{u_features}, u+y_init:{u_features + y_init_features}.")

        # 3. Convert final prepared array to tensor
        if final_np_input is None: raise RuntimeError("Internal error preparing input.")
        input_tensor = torch.from_numpy(final_np_input).float().to(self.device)

        # 4. Run Inference
        output_tensor = None
        with torch.no_grad():
            reset_model_state(self.core_model)
            model_output = self.norm_model(input_tensor) # Main norm applied here
            # Handle tuple outputs if necessary
            if isinstance(model_output, tuple): output_tensor = model_output[0]
            else: output_tensor = model_output

        # 5. Return result
        if output_tensor is None: raise RuntimeError("Model output is None.")
        return output_tensor.squeeze(0).cpu().numpy()

    def __call__(self, np_input: np.ndarray, np_output_init: np.ndarray = None) -> np.ndarray:
        return self.inference(np_input, np_output_init)

In [ ]:
from tsfast.datasets.core import create_dls_test
from tsfast.learner import RNNLearner
from tsfast.prediction import FranSysLearner


In [ ]:
dls = create_dls_test()
lrn = RNNLearner(dls)
model = InferenceWrapper(lrn)

In [ ]:
model(np.random.randn(100, 1)).shape

(100, 1)

In [ ]:
model(np.random.randn(100)).shape

(100, 1)

In [ ]:
model(np.random.randn(1,100,1)).shape

(100, 1)

In [ ]:
lrn = FranSysLearner(dls,10,attach_output=True)
model = InferenceWrapper(lrn)

InferenceWrapper: Detected PredictionCallback(t_offset=0).


In [ ]:
model(np.random.randn(100, 1),np.random.randn(100, 1)).shape

(100, 1)

In [ ]:
lrn = FranSysLearner(dls,10,attach_output=False)
model = InferenceWrapper(lrn)

In [ ]:
model(np.random.randn(100, 1),np.random.randn(100, 1)).shape

/var/folders/pc/13zbh_m514n1tp522cx9npt00000gn/T/ipykernel_5564/249888661.py:90: UserWarning: Input `u` matches expected features. Ignoring provided y_init.
  warnings.warn("Input `u` matches expected features. Ignoring provided y_init.", UserWarning)


(100, 1)

In [ ]:
#| include: false
import nbdev; nbdev.nbdev_export()